In [1]:
# Some configuations
# This cell defines a magic command to ensure that the script doesn't stop due
# to any error arising in that cell.
from IPython.core.magic import register_cell_magic
@register_cell_magic('handle')
def handle(line, cell):
    try:
#         exec(cell)  # doesn't return the cell output though
        return eval(cell)
    except Exception as exc:
        print(f"\033[1;31m{exc.__class__.__name__} : \033[1;31;47m{exc}\033[0m")
        # raise # if you want the full trace-back in the notebook

# Indexing

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv', index_col='ID')
df.head(10)

## (1) [] method

```[]``` method can be used to select column(s) by passing column name(s) (label(s)). Although it also can be used to select row(s), to avoid confusion, we will leave selecting rows to the other two methods, which we will talk about later. 

```df[colname]``` Select single column

In [ ]:
df["Address"].head()

```df[[col1, col2,..., coln]]``` Select multiple columns

In [ ]:
df[["Address", "Math"]].head()

## (2) loc method

**loc** can be used to index row(s) and column(s) by providing the row and column labels.

```df.loc[row_label(s)]``` Selects single row or subset of rows from the DataFrame by label.

In [ ]:
df.loc[1105] # index single row

In [ ]:
df.loc[[1105]] # index single row and keep the dimension

In [ ]:
df.loc[1105:1203] # index multiple rows

In [ ]:
%%handle
df.loc[[1105:1203]] # index multiple rows

In [ ]:
df.loc[[1105, 1102, 1202, 1203]] # index multiple rows

```df.loc[:, col_labels]``` Selects single column or subset of columns by label

In [ ]:
df.loc[:, "School"] # index single column

In [ ]:
df.loc[:, "School":"Weight"].head() # index multiple columns

In [ ]:
df.loc[:, ["School", "Weight", "Address"]].tail() # index multiple columns

```df.loc[row_label(s), col_label(s)]``` Select both rows and columns by label

In [ ]:
df.loc[[1105], ["Address"]]

In [ ]:
df.loc[[1105], "Address"]

In [ ]:
df.loc[1105, ["Address"]]

In [ ]:
df.loc[1105, "Address"]

In [ ]:
df.loc[1105:1205, ["Address", "Math"]]

In [ ]:
df.index

In [ ]:
df.columns

Index by function

In [ ]:
df.head()

In [ ]:
df.loc[lambda x:x['Gender']=='M'].head() # here x stands for each row in df

Index by Boolean Series

In [ ]:
df['Address']

In [ ]:
df['Address'].isin(['street_7','street_4'])

In [ ]:
type(df['Address'].isin(['street_7','street_4']))

In [ ]:
idx = df['Address'].isin(['street_7','street_4'])
df.loc[idx]

In [ ]:
idx = df['Physics'].isin(['B+', 'A', 'A+'])
df.loc[idx]

In [ ]:
df.loc[(df["Physics"] == "A+") & (df["Math"] >= 60)] # use "&" (and), "|" (or)  "~" (not) for Pandas

In [ ]:
df["Physics"] == "A+"

In [ ]:
df["Math"] >= 60

In [ ]:
(df["Physics"] == "A+") & (df["Math"] >= 60)

In [ ]:
df.loc[(df["Gender"] == "F") | (df["Height"] >= 170), ["School", "Class"]]

## (3) iloc method

**iloc** can be used to index row(s) and column(s) by providing the row and column integer(s).

```df.iloc[row_integer(s)]``` Selects single row or subset of rows from the DataFrame by integer position

**Note**: same as indexing for sequence (but different with ```loc```, it is 0 basis and the selection is close to the left and open to the right (the last item is excluded).

In [ ]:
df.iloc[[3]]

In [ ]:
df.iloc[3]

In [ ]:
df.iloc[:4] # 5th item is not selected

```df.iloc[:, col_integer(s)]``` Selects single column or multiple columns from the DataFrame by integer positions

In [ ]:
df.iloc[:, 1:3] # 4th column is not selected

```df.iloc[row_integer(s), col_integer(s)]``` Selects row and columns from the DataFrame by integer positions

In [ ]:
df.iloc[-1::-5, 1:3] 

In [ ]:
list(range(35, 0, -5))

Index by Boolean **values**

When pass Boolean to `iloc` method, you need to first convert the series as values to `iloc`; it does not take Pandas Series 

In [ ]:
%%handle
df.iloc[(df["Physics"] == "A+") & (df["Math"] >= 60)]

In [ ]:
type((df["Physics"] == "A+"))

In [ ]:
type((df["Physics"] == "A+").values)

In [ ]:
df.iloc[(df["Physics"] == "A+").values]

In [ ]:
df.iloc[(df["Physics"] == "A+").values & (df["Math"] >= 60).values]

In [ ]:
df.iloc[((df["Physics"] == "A+") & (df["Math"] >= 60)).values]

# Data processing

In data analysis, we usually do 

* **split**: Split our data into different groups based on some key(s).
* **apply**: Apply some functions to each group.

![split_apply](figures/split_apply.png)

## split

```goupby``` function can be used to split our data into different groups.

### Group by single column

In [ ]:
 df

In [ ]:
grouped_single = df.groupby('School') # key here is School
grouped_single

In [ ]:
grouped_single.get_group('S_1').head() # get specific group

In [ ]:
grouped_single.get_group('S_2').head() # get specific group

In [ ]:
grouped_single.groups

In [ ]:
grouped_single.size()

In [ ]:
grouped_single.ngroups

In [ ]:
grouped_single.first() # first row of each group

### group by numeric column

In [ ]:
mathgroup = df.groupby(lambda x: "Greater than or equal to 60" if df.loc[x, "Math"]>= 60 else "Less than 60")
mathgroup.get_group("Greater than or equal to 60")

In [ ]:
mathgroup.get_group("Less than 60")

#### ```cut``` method

In [ ]:
bins = [0,40,60,80,90,100]
cuts = pd.cut(df['Math'],bins=bins) 
cuts

In [ ]:
df.groupby('cuts', observed=True).groups

In the above code, the `observed` determines whether the grouping process includes only those combinations of categorical levels that appear in your data (observed combinations) or includes all possible combinations of categories, even if some combinations don't appear in the data.

- **`observed=True`**: Group only by observed combinations (appearing in the dataset).
- **`observed=False`**: Group all possible combinations, including ones that may not appear in the dataset.

In [ ]:
df.groupby(cuts, observed=True).size()

### Group by multiple columns

In [ ]:
grouped_mul = df.groupby(['School','Class'])
grouped_mul.size()

In [ ]:
grouped_mul.groups

In [ ]:
grouped_mul.get_group(('S_2','C_4'))

## Aggregation

Aggregation means the data can be aggregated into a single quantity to describe the data, for example: mean, median, std, etc

### ```apply``` method



In [ ]:
grouped_single['Math'].apply(lambda x: x.sum()) # here, x stands for the data from each group

In [ ]:
grouped_mul['Height'].apply(lambda x: x.max()-x.min()) # range for each group

### ```agg``` method

In [ ]:
df.columns

In [ ]:
grouped_single[['Math', 'Height', 'Weight']].agg(['sum','mean','std'])

In [ ]:
grouped_single.agg({'Height': ['sum','mean','std'], 'Weight': ['median', 'mean', 'max']})

In [ ]:
grouped_single["Math"].agg([('your columna name for math sum','sum'),('math_mean','mean')])

In [ ]:
grouped_mul.agg({'Height': [('Height_mean','mean'),('Height_std','std')],
                 'Weight': [('Weight_median','median'),('Weight_min','max')]})

In [ ]:
grouped_mul.agg({'Height': [('range', lambda x: x.max()-x.min())],
                 'Weight': [('IQR', lambda x: x.quantile(0.75)-x.quantile(0.25))]})

### frequency table by ```pd.crosstab``` method

For categorical variables, the frequency table is often used to summarize the data. You can use ```pd.crosstab``` method to generate frequency tables

In [ ]:
df

In [ ]:
pd.crosstab(index = df.Gender, columns='count')

In [ ]:
pd.crosstab(index = "count", columns=df.Class)

In [ ]:
pd.crosstab(index = df.Gender, columns=df.Class)

In [ ]:
pd.crosstab(index = df.Gender, columns=df.Class, margins=True)

#### relative frequency table

In [ ]:
pd.crosstab(index = df.Gender, columns='count', normalize=True)

In [ ]:
pd.crosstab(index = "count", columns=df.Class, normalize=True)

In [ ]:
pd.crosstab(df.Gender, df.Class, margins=True, normalize=False)

In [ ]:
pd.crosstab(df.Gender, df.Class, margins=True, normalize=True)

# Join

When we have two datesets, we can combine them together

## ```assign``` method

```assign``` method can be used when you want to combine column-wise

In [ ]:
maths = [x+10 for x in df['Math']]
heights = [x+10 for x in df['Weight']]
df.assign(new_math=maths, new_height=heights).head()

In [ ]:
df.head() # no change to original df

In [ ]:
df['new_math2']=maths
df.head()

## ```concat``` method

```pd.concat([df1, df2], axis = 0)``` can be used to combine two dataframe either row-wise or column-wise depends on value of **axis**: 

* 0 (default, row-wise)
* 1 (column-wise).

In [ ]:
df1 = pd.DataFrame({'A': ['A0', 'A1'], 'B': ['B0', 'B1']}, index = [0,1])
df1

In [ ]:
df2 = pd.DataFrame({'A': ['A2', 'A3'], 'B': ['B2', 'B3']}, index = [2,3])
df2

In [ ]:
df3 = pd.DataFrame({'A': ['A1', 'A3'], 'B': ['D1', 'D3']}, index = [1,3])
df3

In [ ]:
pd.concat([df1, df2])

In [ ]:
a = pd.concat([df1, df3])
a.loc[1]

In [ ]:
pd.concat([df1, df2], axis = 1)

In [ ]:
df4 = pd.concat([df1, df2], axis = 1)
df4['A']

## ```join``` method

The ```join``` method takes two dataframes and joins them on their indexes

In [108]:
# Dataframe of number of sales made by an employee
sales = pd.DataFrame([103, 202, 380, 101, 82, 99, 500],
                     index = ['Tony', 'Sally', 'Randy','Ellen','Fred', 'Tom', 'HanWei'],
                     columns=['sales'])
# Dataframe of all employees and the region they work in
regions = pd.DataFrame(['West', 'South', 'West', 'North', 'South', 'North', 'East', 'West'],
                     index = ['Tony', 'Sally', 'Carl', 'Archie', 'Randy','Ellen','Fred', 'Mo'],
                     columns=['regions'])

In [ ]:
sales

In [ ]:
regions

### left join (default)

Join based on the table on the indeces of the "left" table. Indeces that are not in the left will not be shown

In [ ]:
sales.join(regions)

### right join 

Join based on the table on the indeces of the "right" table. Indeces that are not in the right will not be shown

In [ ]:
sales.join(regions, how='right')

In [ ]:
regions.join(sales)

### inner join 

Only show indeces if they are available in both table (Think this as intersection operation of set)

In [ ]:
regions.join(sales, how='inner')

### outer join 

Show indeces if they they exist in one table (Think this as union operation of set)

In [ ]:
regions.join(sales, how='outer')